In [1]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.datasets import SVHN
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

learning_rate = 0.001
train_epochs = 100
batch_size=128

In [10]:
#dataset = SVHN(root='data/', download=True, transform=ToTensor())
trainset = SVHN(root='data/', split='train', download=True, transform=ToTensor())
testset = SVHN(root='data/', split='test', download=True, transform=ToTensor())

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [11]:
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)

In [92]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.8
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc1 = torch.nn.Linear(3 * 3 * 128, 1024, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1-self.keep_prob)
        )
        self.fc2 = torch.nn.Linear(1024, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        print("1 ", out.shape)
        out = self.layer2(out)
        print("2 ", out.shape)
        out = self.layer3(out)
        print("3 ", out.shape)
        out.view(out.size(0), -1)
        print("4 ", out.shape)
        out = self.layer4(out)
        print("5 ", out.shape)
        out = self.fc1(out)
        
        return out

In [93]:
model = CNN().to(device)

In [94]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [95]:
total_batch = len(train_loader)
print(f"총 배치의 수 : {total_batch}")

총 배치의 수 : 572


In [96]:
for epoch in range(train_epochs):
    avg_cost = 0
    
    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    print(f"[Epoch: {epoch+1:>4} cost = {avg_cost:>.9}]")

1  torch.Size([128, 32, 15, 15])
2  torch.Size([128, 64, 6, 6])
3  torch.Size([128, 128, 3, 3])
4  torch.Size([128, 128, 3, 3])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (49152x3 and 1152x1024)